Chapter 24 Filtering and Indexing Operations

In [1]:
# deletes variables
%reset -f

In [2]:
import pandas as pd; print(f"– pandas VERSION: {pd.__version__}");


– pandas VERSION: 1.4.4


In [3]:
# version 2 , simplified the columns assignment, combined and modified rename(), move rename() with underscore
def tweak_siena_pres_2(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8' for col in cols}))

    return (df
     .rename(columns=
        {'Seq.':'Seq',
         'Bg': 'Background',
         'PL': 'Party leadership', 'CAb': 'Communication ability',
         'RC': 'Relations with Congress', 'CAp': 'Court appointments',
         'HE': 'Handling of economy', 'L': 'Luck',
         'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
         'EAp': 'Executive appointments', 'OA': 'Overall ability',
         'Im': 'Imagination', 'DA': 'Domestic accomplishments',
         'Int': 'Integrity', 'EAb': 'Executive ability',
         'FPA': 'Foreign policy accomplishments',
         'LA': 'Leadership ability',
         'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
         'EV': "Experts' view", 'O': 'Overall'})
     .rename(columns = lambda c:c.replace(' ','_'))
     .astype({'Party':'category'})  # 2
     .pipe(int64_to_uint8)  # 3
     .assign(Average_rank=lambda df_:df_.select_dtypes('uint8') # 4
                 .sum(axis=1).rank(method='dense').astype('uint8'),
             Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                 labels='1st 2nd 3rd 4th'.split())
            )
    )

In [4]:
pd.set_option('display.max_rows', 10)             
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
    'siena2018-pres.csv'
df = pd.read_csv(url, index_col=0)

In [5]:
pres = tweak_siena_pres_2(df)

In [6]:
def name_to_initial(val: str):
    """Input a pandas series of 
    names and out put first initial and last name"""
    names = val.split()
    return ' '.join([f'{names[0][0]}.',*names[1:]]) 
    

In [7]:
(pres
 .set_index('President')
 .rename(name_to_initial))


,Seq,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
President,,,,,,,,,,,,,,,,,,,,,
G. Washington,1,Independent,7,7,1,10,1,6,2,2,...,1,1,1,2,2,1,2,1,1,1st
J. Adams,2,Federalist,3,13,4,4,24,14,31,21,...,4,13,15,19,13,16,10,14,13,2nd
T. Jefferson,3,Democratic-Republican,2,2,14,1,8,5,14,6,...,7,20,4,6,9,7,5,5,5,1st
J. Madison,4,Democratic-Republican,4,6,7,3,16,15,6,13,...,6,14,7,11,19,11,8,7,7,1st
J. Monroe,5,Democratic-Republican,9,14,11,18,6,16,7,10,...,11,9,9,10,5,6,9,8,8,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G. H. W. Bush,41,Republican,10,27,18,19,20,27,13,20,...,29,28,19,26,10,17,21,21,21,2nd
B. Clinton,42,Democratic,21,12,39,8,11,17,3,16,...,10,5,12,9,18,30,14,15,15,2nd
G. W. Bush,43,Republican,17,29,33,41,21,20,28,34,...,28,36,29,30,38,36,34,33,33,3rd


In [9]:
#24.2 Resetting the index
(pres
 .set_index('President')
 .reset_index()
)

,President,Seq,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
0,George Washington,1,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st
1,John Adams,2,Federalist,3,13,4,4,24,14,31,...,4,13,15,19,13,16,10,14,13,2nd
2,Thomas Jefferson,3,Democratic-Republican,2,2,14,1,8,5,14,...,7,20,4,6,9,7,5,5,5,1st
3,James Madison,4,Democratic-Republican,4,6,7,3,16,15,6,...,6,14,7,11,19,11,8,7,7,1st
4,James Monroe,5,Democratic-Republican,9,14,11,18,6,16,7,...,11,9,9,10,5,6,9,8,8,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,George H. W. Bush,41,Republican,10,27,18,19,20,27,13,...,29,28,19,26,10,17,21,21,21,2nd
40,Bill Clinton,42,Democratic,21,12,39,8,11,17,3,...,10,5,12,9,18,30,14,15,15,2nd
41,George W. Bush,43,Republican,17,29,33,41,21,20,28,...,28,36,29,30,38,36,34,33,33,3rd
42,Barack Obama,44,Democratic,24,11,13,9,15,23,16,...,14,10,13,13,20,10,11,17,17,2nd


In [10]:
#24.3 Dataframe Indexing, Filtering & Querying

lt10 = pres.Average_rank < 10
pres[lt10]


,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,...,7,20,4,6,9,7,5,5,5,1st
4,4,James Madison,Democratic-Republican,4,6,7,3,16,15,6,...,6,14,7,11,19,11,8,7,7,1st
5,5,James Monroe,Democratic-Republican,9,14,11,18,6,16,7,...,11,9,9,10,5,6,9,8,8,1st
16,16,Abraham Lincoln,Republican,28,1,2,2,18,1,1,...,3,4,2,1,6,2,1,3,3,1st
25,26,Theodore Roosevelt,Republican,5,4,8,6,2,2,15,...,9,3,5,4,3,5,4,4,4,1st
31,32,Franklin D. Roosevelt,Democratic,6,3,16,12,5,3,4,...,2,2,3,3,1,4,3,2,2,1st
32,33,Harry S. Truman,Democratic,31,16,9,21,12,8,12,...,17,8,10,7,4,9,7,9,9,1st
33,34,Dwight D. Eisenhower,Republican,11,18,5,17,7,21,5,...,5,6,11,8,7,3,6,6,6,1st


In [ ]:
pres[lt10 & ( pres.Party == 'Republican')]

In [ ]:
pres[(pres.Average_rank < 10 )  & (pres.Party == 'Republican')]

In [15]:
pres.query('Average_rank < 10 and Party == "Republican"')

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
16,16,Abraham Lincoln,Republican,28,1,2,2,18,1,1,...,3,4,2,1,6,2,1,3,3,1st
25,26,Theodore Roosevelt,Republican,5,4,8,6,2,2,15,...,9,3,5,4,3,5,4,4,4,1st
33,34,Dwight D. Eisenhower,Republican,11,18,5,17,7,21,5,...,5,6,11,8,7,3,6,6,6,1st


In [ ]:
lt10 = pres.Average_rank < 10

pres.query('@lt10 and Party == "Republican"')

In [16]:
# 24.4 Indexing by Position

pres.iloc[1]

Seq                                2
President                 John Adams
Party                     Federalist
Background                         3
Imagination                       13
                             ...    
Avoid_crucial_mistakes            16
Experts'_view                     10
Overall                           14
Average_rank                      13
Quartile                         2nd
Name: 2, Length: 26, dtype: object

In [17]:
pres.iloc[[1]]

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
2,2,John Adams,Federalist,3,13,4,4,24,14,31,...,4,13,15,19,13,16,10,14,13,2nd


In [18]:
pres.iloc[1]

Seq                                2
President                 John Adams
Party                     Federalist
Background                         3
Imagination                       13
                             ...    
Avoid_crucial_mistakes            16
Experts'_view                     10
Overall                           14
Average_rank                      13
Quartile                         2nd
Name: 2, Length: 26, dtype: object

In [19]:
pres.iloc[[0,5,10]]

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st
6,6,John Quincy Adams,Democratic-Republican,1,9,6,5,29,19,24,...,15,17,18,21,15,14,18,18,18,2nd
11,11,James K. Polk,Democratic,19,10,23,23,9,7,18,...,22,15,16,12,8,8,13,12,11,1st


In [20]:
pres.iloc[0:11:5]

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st
6,6,John Quincy Adams,Democratic-Republican,1,9,6,5,29,19,24,...,15,17,18,21,15,14,18,18,18,2nd
11,11,James K. Polk,Democratic,19,10,23,23,9,7,18,...,22,15,16,12,8,8,13,12,11,1st


In [21]:
pres.iloc[[0,5,10]]

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st
6,6,John Quincy Adams,Democratic-Republican,1,9,6,5,29,19,24,...,15,17,18,21,15,14,18,18,18,2nd
11,11,James K. Polk,Democratic,19,10,23,23,9,7,18,...,22,15,16,12,8,8,13,12,11,1st


In [22]:
pres.iloc[lambda df: [0,5,10]]

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st
6,6,John Quincy Adams,Democratic-Republican,1,9,6,5,29,19,24,...,15,17,18,21,15,14,18,18,18,2nd
11,11,James K. Polk,Democratic,19,10,23,23,9,7,18,...,22,15,16,12,8,8,13,12,11,1st


In [24]:
pres.iloc[[0,5,10],1]

1     George Washington
6     John Quincy Adams
11        James K. Polk
Name: President, dtype: object

In [25]:
pres.iloc[[5,5,10],[1]]

,President
6,John Quincy Adams
6,John Quincy Adams
11,James K. Polk


In [26]:
pres.iloc[:,[1,2]]

,President,Party
1,George Washington,Independent
2,John Adams,Federalist
3,Thomas Jefferson,Democratic-Republican
4,James Madison,Democratic-Republican
5,James Monroe,Democratic-Republican
...,...,...
40,George H. W. Bush,Republican
41,Bill Clinton,Democratic
42,George W. Bush,Republican
43,Barack Obama,Democratic


In [27]:
pres.iloc[:, 1:3]

,President,Party
1,George Washington,Independent
2,John Adams,Federalist
3,Thomas Jefferson,Democratic-Republican
4,James Madison,Democratic-Republican
5,James Monroe,Democratic-Republican
...,...,...
40,George H. W. Bush,Republican
41,Bill Clinton,Democratic
42,George W. Bush,Republican
43,Barack Obama,Democratic


In [28]:
# Indexing by name
# need to set the index to display error.

pres2 = pres.set_index('Seq')
pres2.loc[1:5]

TypeError: cannot do slice indexing on Index with these indexers [1] of type int

In [29]:
pres2.loc['1':'5']

,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
Seq,,,,,,,,,,,,,,,,,,,,,
1,George Washington,Independent,7,7,1,10,1,6,2,2,...,1,1,1,2,2,1,2,1,1,1st
2,John Adams,Federalist,3,13,4,4,24,14,31,21,...,4,13,15,19,13,16,10,14,13,2nd
3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,6,...,7,20,4,6,9,7,5,5,5,1st
4,James Madison,Democratic-Republican,4,6,7,3,16,15,6,13,...,6,14,7,11,19,11,8,7,7,1st
5,James Monroe,Democratic-Republican,9,14,11,18,6,16,7,10,...,11,9,9,10,5,6,9,8,8,1st


In [ ]:
pres.iloc[1:5]

In [30]:
# the return is a dataframe because there are multiple rows.
(pres
 .set_index('Party')
 .loc['Whig'])

,Seq,President,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
Party,,,,,,,,,,,,,,,,,,,,,
Whig,9,William Henry Harrison,22,38,28,37,44,32,41,38,...,42,41,40,42,44,37,39,39,38,4th
Whig,12,Zachary Taylor,30,26,22,32,37,24,26,26,...,37,27,33,27,30,26,30,30,30,3rd
Whig,13,Millard Fillmore,40,37,36,38,35,38,32,37,...,39,37,37,37,37,33,37,38,39,4th


In [31]:
# the return is a dataframe
(pres
 .set_index('Party')
 .loc[['Whig']])

,Seq,President,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
Party,,,,,,,,,,,,,,,,,,,,,
Whig,9,William Henry Harrison,22,38,28,37,44,32,41,38,...,42,41,40,42,44,37,39,39,38,4th
Whig,12,Zachary Taylor,30,26,22,32,37,24,26,26,...,37,27,33,27,30,26,30,30,30,3rd
Whig,13,Millard Fillmore,40,37,36,38,35,38,32,37,...,39,37,37,37,37,33,37,38,39,4th


In [32]:
# returns a series instead of dataframe because there is only one line
(pres
 .set_index('Party')
 .loc['Federalist'])

Seq                                2
President                 John Adams
Background                         3
Imagination                       13
Integrity                          4
                             ...    
Avoid_crucial_mistakes            16
Experts'_view                     10
Overall                           14
Average_rank                      13
Quartile                         2nd
Name: Federalist, Length: 25, dtype: object

In [33]:
# slice with string indexes
# must sort 
# can slice with partial values

(pres
 .set_index('Party')
 .loc['Democratic':'Independent'])

KeyError: "Cannot get left slice bound for non-unique label: 'Democratic'"

In [34]:
(pres
 .set_index('Party')
 .sort_index()
 .loc['Democratic':'Independent'])

,Seq,President,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
Party,,,,,,,,,,,,,,,,,,,,,
Democratic,22/24,Grover Cleveland,26,23,26,27,19,27,22,19,...,20,21,23,23,21,15,22,23,23,3rd
Democratic,32,Franklin D. Roosevelt,6,3,16,12,5,3,4,3,...,2,2,3,3,1,4,3,2,2,1st
Democratic,17,Andrew Johnson,42,42,41,42,40,34,43,43,...,44,43,42,43,41,43,43,44,44,4th
Democratic,33,Harry S. Truman,31,16,9,21,12,8,12,8,...,17,8,10,7,4,9,7,9,9,1st
Democratic,15,James Buchanan,36,43,40,39,42,41,40,42,...,43,42,43,44,43,44,44,43,43,4th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Democratic-Republican,4,James Madison,4,6,7,3,16,15,6,13,...,6,14,7,11,19,11,8,7,7,1st
Democratic-Republican,3,Thomas Jefferson,2,2,14,1,8,5,14,6,...,7,20,4,6,9,7,5,5,5,1st
Federalist,2,John Adams,3,13,4,4,24,14,31,21,...,4,13,15,19,13,16,10,14,13,2nd


In [36]:
(pres
 .set_index('President')
 .sort_index()
 .loc['C':'Thomas Jefferson','Party':'Integrity']
)

,Party,Background,Imagination,Integrity
President,,,,
Calvin Coolidge,Republican,32,36,17
Chester A. Arthur,Republican,41,31,37
Donald Trump,Republican,43,40,44
Dwight D. Eisenhower,Republican,11,18,5
Franklin D. Roosevelt,Democratic,6,3,16
...,...,...,...,...
Richard Nixon,Republican,16,21,43
Ronald Reagan,Republican,27,17,24
Rutherford B. Hayes,Republican,35,30,32


In [37]:
(pres
 .set_index('Party')
 .sort_index()
 .loc['D':'J'])

TypeError: Cannot setitem on a Categorical with a new category (D), set the categories first

In [40]:
(pres
 .assign(Party=pres.Party.astype(str))
 .set_index('Party')
 .sort_index()
 .loc['D':'J'])

,Seq,President,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
Party,,,,,,,,,,,,,,,,,,,,,
Democratic,22/24,Grover Cleveland,26,23,26,27,19,27,22,19,...,20,21,23,23,21,15,22,23,23,3rd
Democratic,32,Franklin D. Roosevelt,6,3,16,12,5,3,4,3,...,2,2,3,3,1,4,3,2,2,1st
Democratic,17,Andrew Johnson,42,42,41,42,40,34,43,43,...,44,43,42,43,41,43,43,44,44,4th
Democratic,33,Harry S. Truman,31,16,9,21,12,8,12,8,...,17,8,10,7,4,9,7,9,9,1st
Democratic,15,James Buchanan,36,43,40,39,42,41,40,42,...,43,42,43,44,43,44,44,43,43,4th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Democratic-Republican,4,James Madison,4,6,7,3,16,15,6,13,...,6,14,7,11,19,11,8,7,7,1st
Democratic-Republican,3,Thomas Jefferson,2,2,14,1,8,5,14,6,...,7,20,4,6,9,7,5,5,5,1st
Federalist,2,John Adams,3,13,4,4,24,14,31,21,...,4,13,15,19,13,16,10,14,13,2nd


In [42]:
(pres
 .set_index('President')
 .sort_index()
 .sort_index(axis='columns')
 .loc['C':'Thomas Jefferson','B':'D']
)

,Background,Communication_ability,Court_appointments
President,,,
Calvin Coolidge,32,37,31
Chester A. Arthur,41,36,33
Donald Trump,43,43,40
Dwight D. Eisenhower,11,20,5
Franklin D. Roosevelt,6,2,2
...,...,...,...
Richard Nixon,16,27,32
Ronald Reagan,27,6,18
Rutherford B. Hayes,35,30,27


In [44]:
(pres
 .loc[pres.Average_rank < 10, lambda df_: df_.columns[:3]])

,Seq,President,Party
1,1,George Washington,Independent
3,3,Thomas Jefferson,Democratic-Republican
4,4,James Madison,Democratic-Republican
5,5,James Monroe,Democratic-Republican
16,16,Abraham Lincoln,Republican
25,26,Theodore Roosevelt,Republican
31,32,Franklin D. Roosevelt,Democratic
32,33,Harry S. Truman,Democratic
33,34,Dwight D. Eisenhower,Republican
